# Configuration

> Project-level configuration for paths, runtime settings, and environment management

In [ ]:
#| default_exp core.config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import platform as platform_mod
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
from typing import Dict, Optional

import yaml

## Configuration Enums

Enums for runtime mode and conda implementation type.

In [ ]:
#| export
class RuntimeMode(str, Enum):
    """Runtime mode for the plugin system."""
    LOCAL = "local"    # Project-local runtime and data
    SYSTEM = "system"  # System-wide conda and ~/.cjm data


class CondaType(str, Enum):
    """Type of conda implementation to use."""
    MICROMAMBA = "micromamba"
    MINIFORGE = "miniforge"
    CONDA = "conda"

## Configuration Dataclasses

`RuntimeConfig` holds conda/environment settings. `CJMConfig` is the main configuration container with paths and runtime settings.

In [ ]:
#| export
@dataclass
class RuntimeConfig:
    """Runtime environment configuration."""
    mode:RuntimeMode=RuntimeMode.SYSTEM # LOCAL for project-local, SYSTEM for global
    conda_type:CondaType=CondaType.CONDA # Conda implementation to use
    prefix:Optional[Path]=None # Path to runtime directory (LOCAL mode only)
    binaries:Dict[str, Path]=field(default_factory=dict) # Platform-specific binary paths

In [ ]:
#| export
@dataclass
class CJMConfig:
    """Main configuration for cjm-plugin-system."""
    runtime:RuntimeConfig=field(default_factory=RuntimeConfig) # Runtime environment settings
    data_dir:Path=field(default_factory=lambda: Path.home() / ".cjm") # Base directory for manifests, logs
    plugins_config:Path=field(default_factory=lambda: Path("plugins.yaml")) # Path to plugins.yaml file
    models_dir:Optional[Path]=None # Directory for model downloads

    @property
    def plugins_dir(self) -> Path: # Directory containing plugin manifests
        """Directory containing plugin manifests."""
        return self.data_dir / "plugins"

    @property
    def logs_dir(self) -> Path: # Directory containing plugin logs
        """Directory containing plugin logs."""
        return self.data_dir / "logs"

    @property
    def conda_binary_path(self) -> Optional[Path]: # Path to conda/micromamba binary or None
        """Get the configured binary path for the current platform."""
        # Inline platform detection to avoid circular imports
        system = platform_mod.system().lower()
        machine = platform_mod.machine().lower()
        
        if system == "windows":
            system = "win"
        if machine in ("x86_64", "amd64"):
            arch = "x64"
        elif machine in ("arm64", "aarch64"):
            arch = "arm64"
        else:
            arch = machine
        
        platform_key = f"{system}-{arch}"
        
        if self.runtime.binaries and platform_key in self.runtime.binaries:
            return self.runtime.binaries[platform_key]
        
        # Default location if prefix is set
        if self.runtime.prefix:
            binary_name = "micromamba.exe" if system == "win" else "micromamba"
            return self.runtime.prefix / "bin" / binary_name
        
        return None

## Configuration Loading

Functions for loading configuration with layered resolution:

1. CLI flags (highest priority)
2. Environment variables
3. `cjm.yaml` file
4. Defaults (backward compatible)

In [ ]:
#| export
# Module-level configuration singleton
_current_config: Optional[CJMConfig] = None

In [ ]:
#| export
def _load_from_yaml(
    yaml_path:Path # Path to cjm.yaml file
) -> CJMConfig: # Parsed configuration
    """Load config from YAML file, resolving relative paths."""
    with open(yaml_path) as f:
        data = yaml.safe_load(f) or {}

    # Resolve relative paths against yaml file location
    base_dir = yaml_path.parent.resolve()

    # Parse runtime config
    runtime_data = data.get("runtime", {})
    runtime = RuntimeConfig(
        mode=RuntimeMode(runtime_data.get("mode", "system")),
        conda_type=CondaType(runtime_data.get("conda_type", "conda")),
        prefix=base_dir / runtime_data["prefix"] if runtime_data.get("prefix") else None,
        binaries={k: base_dir / v for k, v in runtime_data.get("binaries", {}).items()}
    )

    # Parse top-level config
    config = CJMConfig(runtime=runtime)

    if "data_dir" in data:
        config.data_dir = base_dir / data["data_dir"]
    if "plugins_config" in data:
        config.plugins_config = base_dir / data["plugins_config"]
    if "models_dir" in data:
        config.models_dir = base_dir / data["models_dir"]

    return config

In [ ]:
#| export
def load_config(
    config_path:Optional[Path]=None, # CLI --cjm-config
    data_dir:Optional[Path]=None, # CLI --data-dir
    conda_prefix:Optional[Path]=None, # CLI --conda-prefix
    conda_type:Optional[str]=None # CLI --conda-type
) -> CJMConfig: # Resolved configuration
    """Load config with layered resolution (CLI > env vars > yaml > defaults)."""
    # 1. Start with defaults
    config = CJMConfig()

    # 2. Load cjm.yaml if exists (specified or in cwd)
    yaml_path = config_path or Path("cjm.yaml")
    if yaml_path.exists():
        config = _load_from_yaml(yaml_path)

    # 3. Override with environment variables
    if env_data_dir := os.environ.get("CJM_DATA_DIR"):
        config.data_dir = Path(env_data_dir)
    if env_conda_prefix := os.environ.get("CJM_CONDA_PREFIX"):
        config.runtime.prefix = Path(env_conda_prefix)
    if env_conda_type := os.environ.get("CJM_CONDA_TYPE"):
        config.runtime.conda_type = CondaType(env_conda_type)

    # 4. Override with CLI args (highest priority)
    if data_dir:
        config.data_dir = data_dir
    if conda_prefix:
        config.runtime.prefix = conda_prefix
    if conda_type:
        config.runtime.conda_type = CondaType(conda_type)

    return config

## Global Config Access

Functions for getting and setting the module-level configuration singleton.

In [ ]:
#| export
def get_config() -> CJMConfig: # Current configuration
    """Get current config (loads defaults if not set)."""
    global _current_config
    if _current_config is None:
        _current_config = load_config()
    return _current_config


def set_config(
    config:CJMConfig # Configuration to set as current
) -> None:
    """Set current config (called by CLI callback)."""
    global _current_config
    _current_config = config


def reset_config() -> None:
    """Reset to unloaded state (for testing)."""
    global _current_config
    _current_config = None

## Examples

In [ ]:
# Test default configuration
reset_config()
cfg = get_config()

print("Default configuration:")
print(f"  data_dir: {cfg.data_dir}")
print(f"  plugins_dir: {cfg.plugins_dir}")
print(f"  logs_dir: {cfg.logs_dir}")
print(f"  plugins_config: {cfg.plugins_config}")
print(f"  runtime.mode: {cfg.runtime.mode}")
print(f"  runtime.conda_type: {cfg.runtime.conda_type}")

Default configuration:
  data_dir: /home/innom-dt/.cjm
  plugins_dir: /home/innom-dt/.cjm/plugins
  logs_dir: /home/innom-dt/.cjm/logs
  plugins_config: plugins.yaml
  runtime.mode: RuntimeMode.SYSTEM
  runtime.conda_type: CondaType.CONDA


In [ ]:
# Test CLI override
reset_config()
cfg = load_config(data_dir=Path("/custom/path"))

print("With CLI override:")
print(f"  data_dir: {cfg.data_dir}")
print(f"  plugins_dir: {cfg.plugins_dir}")

With CLI override:
  data_dir: /custom/path
  plugins_dir: /custom/path/plugins


In [ ]:
# Test dataclass creation
runtime = RuntimeConfig(
    mode=RuntimeMode.LOCAL,
    conda_type=CondaType.MINIFORGE,
    prefix=Path("./runtime")
)

config = CJMConfig(
    runtime=runtime,
    data_dir=Path("./.cjm")
)

print("Custom configuration:")
print(f"  runtime.mode: {config.runtime.mode}")
print(f"  runtime.conda_type: {config.runtime.conda_type}")
print(f"  runtime.prefix: {config.runtime.prefix}")
print(f"  data_dir: {config.data_dir}")

Custom configuration:
  runtime.mode: RuntimeMode.LOCAL
  runtime.conda_type: CondaType.MINIFORGE
  runtime.prefix: runtime
  data_dir: .cjm


In [ ]:
# Test conda_binary_path property
runtime_with_binaries = RuntimeConfig(
    conda_type=CondaType.MICROMAMBA,
    mode=RuntimeMode.LOCAL,
    prefix=Path("./runtime"),
    binaries={"linux-x64": Path("./runtime/bin/micromamba")}
)

config_with_binaries = CJMConfig(runtime=runtime_with_binaries)
print(f"conda_binary_path (from binaries): {config_with_binaries.conda_binary_path}")

# Test default path generation when binaries not specified
runtime_no_binaries = RuntimeConfig(
    conda_type=CondaType.MICROMAMBA,
    mode=RuntimeMode.LOCAL,
    prefix=Path("./runtime")
)

config_no_binaries = CJMConfig(runtime=runtime_no_binaries)
print(f"conda_binary_path (default): {config_no_binaries.conda_binary_path}")

conda_binary_path (from binaries): runtime/bin/micromamba
conda_binary_path (default): runtime/bin/micromamba


In [ ]:
# Cleanup
reset_config()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()